In [15]:
!pip -q install pandas==2.2.2 numpy==2.0.2 scikit-learn==1.6.1 joblib==1.5.3 openpyxl==3.1.2

import pandas as pd
import numpy as np
import joblib
import glob, os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

print("✅ Ready")

✅ Ready


In [16]:
from google.colab import files
uploaded = files.upload()
print("Uploaded:", list(uploaded.keys()))

Saving FD_Deposit_Master (1).csv to FD_Deposit_Master (1) (2).csv
Saving FD_DepositEntries (1).csv to FD_DepositEntries (1) (2).csv
Saving FD_DepositEntries2 (1).csv to FD_DepositEntries2 (1) (2).csv
Saving FD_History_DepositMaster  (1).xlsx to FD_History_DepositMaster  (1) (2).xlsx
Saving Loan_Dataset_Synthetic.xlsx to Loan_Dataset_Synthetic (2).xlsx
Saving NICs and CustomerDetails are added to FDs and SAs.xlsx to NICs and CustomerDetails are added to FDs and SAs (2).xlsx
Saving NICs are added to FDs and SAs.xlsx to NICs are added to FDs and SAs (2).xlsx
Uploaded: ['FD_Deposit_Master (1) (2).csv', 'FD_DepositEntries (1) (2).csv', 'FD_DepositEntries2 (1) (2).csv', 'FD_History_DepositMaster\u202f (1) (2).xlsx', 'Loan_Dataset_Synthetic (2).xlsx', 'NICs and CustomerDetails are added to FDs and SAs (2).xlsx', 'NICs are added to FDs and SAs (2).xlsx']


In [17]:
def find_file(patterns):
    for p in patterns:
        m = glob.glob(p)
        if m:
            return sorted(m)[-1]
    return None

cust_xlsx = find_file(["CustomerDetails.xlsx", "CustomerDetails.xls", "NICs and CustomerDetails*.xlsx", "NICs and CustomerDetails*.xls"])
fd_csv    = find_file(["FD_Deposit_Master*.csv", "FD_Deposit_Master*.CSV"])
loan_xlsx = find_file(["Loan_Dataset*.xlsx", "Loan*.xlsx"])

print("Customer file:", cust_xlsx)
print("FD master:", fd_csv)
print("Loan file:", loan_xlsx)

assert cust_xlsx is not None, "❌ Upload NICs and CustomerDetails.xlsx"
assert fd_csv is not None, "❌ Upload FD_Deposit_Master.csv"
assert loan_xlsx is not None, "❌ Upload Loan_Dataset.xlsx"

xls = pd.ExcelFile(cust_xlsx)
print("Customer sheets:", xls.sheet_names)

# Prefer a sheet containing 'customer'
cust_sheet = None
for s in xls.sheet_names:
    if "customer" in s.lower():
        cust_sheet = s
        break
if cust_sheet is None:
    cust_sheet = xls.sheet_names[0]

cust = pd.read_excel(xls, sheet_name=cust_sheet)
fd   = pd.read_csv(fd_csv)
loan = pd.read_excel(loan_xlsx)

cust.head(), fd.head(), loan.head()

Customer file: NICs and CustomerDetails are added to FDs and SAs.xlsx
FD master: FD_Deposit_Master (1).csv
Loan file: Loan_Dataset_Synthetic.xlsx
Customer sheets: ['FD_NIC', 'NIC', 'SA_NIC', 'CustomerDetails', 'Customer Details Savings']


(      NIC            Name  Age                             Address  \
 0  NIC001  PZU Jayasinghe   36  No  420, Temple Street, Batticaloa   
 1  NIC002     GYU Bandara   68    No  976, Station Road, Ratnapura   
 2  NIC003  VBD Disanayake   49         No  895, King Street, Galle   
 3  NIC004     RUS Bandara   63     No  144, Victoria Street, Kandy   
 4  NIC005     XJN Bandara   67          No  58, Main Street, Kandy   
 
           Town  
 0   Batticaloa  
 1    Ratnapura  
 2        Galle  
 3        Kandy  
 4        Kandy  ,
   ActDate  Activated    Amount AmountReceived         AppDate  Approved  \
 0     NaN      False   10000.0            $0   4/18/2022 0:00     False   
 1     NaN      False   10000.0            $0   8/12/2024 0:00     False   
 2     NaN      False   42000.0            $0   5/14/2024 0:00     False   
 3     NaN      False   60000.0            $0    9/3/2021 0:00     False   
 4     NaN      False  100000.0            $0    4/2/2021 0:00     False   
 
   Ap

In [20]:
def norm_nic(x):
    return str(x).strip()

def pick_col(df, candidates):
    m = {str(c).strip().lower(): c for c in df.columns}
    for c in candidates:
        if c.strip().lower() in m:
            return m[c.strip().lower()]
    return None

# Ensure join keys are string type for robust merging
fd['FDNo'] = fd['FDNo'].astype(str)
fd_nic['FD_No'] = fd_nic['FD_No'].astype(str)

# Merge fd with fd_nic to add the 'NIC' column
fd = pd.merge(fd, fd_nic, left_on='FDNo', right_on='FD_No', how='left', suffixes=('', '_fd_nic'))
# Drop the redundant FD_No column from the merged dataframe, if it exists and is not the key
if 'FD_No' in fd.columns:
    fd.drop(columns=['FD_No'], inplace=True)

# Detect NIC columns
CUST_NIC = pick_col(cust, ["NIC", "NationalID", "National ID"])
FD_NIC   = pick_col(fd,   ["NIC", "NationalID", "National ID"])
LOAN_NIC = pick_col(loan, ["NIC", "NationalID", "National ID"])

assert CUST_NIC is not None, "❌ Cannot find NIC column in customer file."
assert FD_NIC is not None, "❌ Cannot find NIC column in FD file."
assert LOAN_NIC is not None, "❌ Cannot find NIC column in Loan file."

# Detect other columns
AGE_COL   = pick_col(cust, ["Age", "CustomerAge", "Customer Age"])
TOWN_COL  = pick_col(cust, ["Town", "City", "Location"])

FD_AMT    = pick_col(fd,   ["Amount", "FD_Amount", "FD Amount", "DepositAmount", "Deposit Amount"])
FD_RATE   = pick_col(fd,   ["InterestRate", "Interest Rate", "Rate"])

LOAN_AMT  = pick_col(loan, ["LoanAmount", "Loan Amount", "RequiredLoanAmount", "Loan_Amount_LKR"])
LOAN_RATE = pick_col(loan, ["InterestRate", "Interest Rate", "Rate"])
SALARY    = pick_col(loan, ["MonthlySalary", "Salary", "Monthly Salary"])

assert FD_AMT is not None, "❌ Cannot find FD amount column in FD file."
assert LOAN_AMT is not None, "❌ Cannot find Loan amount column in Loan file."

# Normalize NIC strings
cust[CUST_NIC] = cust[CUST_NIC].astype(str).map(norm_nic)
fd[FD_NIC]     = fd[FD_NIC].astype(str).map(norm_nic)
loan[LOAN_NIC] = loan[LOAN_NIC].astype(str).map(norm_nic)

rows = []
for nic in cust[CUST_NIC].dropna().unique():
    c = cust[cust[CUST_NIC] == nic].head(1)
    age  = int(c[AGE_COL].iloc[0]) if AGE_COL and not c.empty and pd.notna(c[AGE_COL].iloc[0]) else 0
    town = str(c[TOWN_COL].iloc[0]) if TOWN_COL and not c.empty and pd.notna(c[TOWN_COL].iloc[0]) else "Unknown"

    fd_rows = fd[fd[FD_NIC] == nic]
    fd_total = float(pd.to_numeric(fd_rows[FD_AMT], errors="coerce").fillna(0).sum()) if not fd_rows.empty else 0.0
    fd_avg_rate = float(pd.to_numeric(fd_rows[FD_RATE], errors="coerce").fillna(0).mean()) if (not fd_rows.empty and FD_RATE) else 0.0

    loan_rows = loan[loan[LOAN_NIC] == nic]
    loan_total = float(pd.to_numeric(loan_rows[LOAN_AMT], errors="coerce").fillna(0).sum()) if not loan_rows.empty else 0.0
    loan_avg_rate = float(pd.to_numeric(loan_rows[LOAN_RATE], errors="coerce").fillna(0).mean()) if (not loan_rows.empty and LOAN_RATE) else 0.0
    salary = float(pd.to_numeric(loan_rows[SALARY], errors="coerce").fillna(0).max()) if (not loan_rows.empty and SALARY) else 0.0

    rows.append({
        "NIC": nic,
        "Age": age,
        "Town": town,
        "FD_Total_Amount": fd_total,
        "FD_Avg_Rate": fd_avg_rate,
        "Loan_Total": loan_total,
        "Loan_Avg_Rate": loan_avg_rate,
        "Salary": salary
    })

data = pd.DataFrame(rows)
print("✅ Training rows:", len(data))
data.head()

✅ Training rows: 32064


,NIC,Age,Town,FD_Total_Amount,FD_Avg_Rate,Loan_Total,Loan_Avg_Rate,Salary
0,NIC001,36,Batticaloa,10000.0,0.0,0.0,0.0,0.0
1,NIC002,68,Ratnapura,10000.0,0.0,0.0,0.0,0.0
2,NIC003,49,Galle,42000.0,0.0,0.0,0.0,0.0
3,NIC004,63,Kandy,160000.0,0.0,0.0,0.0,0.0
4,NIC005,67,Kandy,0.0,0.0,0.0,0.0,0.0


In [21]:
def infer_came_for(r):
    if r["FD_Total_Amount"] > 0 and r["Loan_Total"] == 0:
        return "FD"
    if r["Loan_Total"] > 0 and r["FD_Total_Amount"] == 0:
        return "Loan"
    if r["Loan_Total"] > 0 and r["FD_Total_Amount"] > 0:
        return "FD+Loan"
    return "New/Other"

data["Customer_Came_For"] = data.apply(infer_came_for, axis=1)

def define_target(r):
    if r["Customer_Came_For"] == "FD":
        if r["FD_Total_Amount"] >= 500000:
            return "Wealth/Investment"
        return "Fixed Deposit"

    if r["Customer_Came_For"] == "FD+Loan":
        return "FD-Secured Loan"

    if r["Customer_Came_For"] == "Loan":
        if r["Salary"] >= 150000:
            return "Credit Card"
        return "Personal Loan"

    return "Savings Account"

data["Next_Best_Product"] = data.apply(define_target, axis=1)

print("✅ Target distribution:")
print(data["Next_Best_Product"].value_counts())
print("\n✅ Came for distribution:")
print(data["Customer_Came_For"].value_counts())

✅ Target distribution:
Next_Best_Product
Savings Account      30529
Wealth/Investment      902
Fixed Deposit          633
Name: count, dtype: int64

✅ Came for distribution:
Customer_Came_For
New/Other    30529
FD            1535
Name: count, dtype: int64


In [22]:
X = data.drop(columns=["NIC", "Next_Best_Product"])
X = pd.get_dummies(X, columns=["Town", "Customer_Came_For"], dummy_na=False)
y = data["Next_Best_Product"]

# Safety
if y.nunique() < 2:
    raise ValueError("❌ Only 1 class found in target. Check your FD/Loan linking & NIC columns.")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = RandomForestClassifier(
    n_estimators=500,
    random_state=42,
    class_weight="balanced",
    min_samples_leaf=2
)
model.fit(X_train, y_train)

print("✅ Accuracy:", model.score(X_test, y_test))
print("✅ Classes:", model.classes_)

joblib.dump(model, "nbp_model.pkl")
joblib.dump(list(X.columns), "input_features.pkl")

print("✅ Saved nbp_model.pkl + input_features.pkl")
print("✅ Feature count:", len(X.columns))

✅ Accuracy: 1.0
✅ Classes: ['Fixed Deposit' 'Savings Account' 'Wealth/Investment']
✅ Saved nbp_model.pkl + input_features.pkl
✅ Feature count: 18


In [23]:
from google.colab import files
files.download("nbp_model.pkl")
files.download("input_features.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>